In [ ]:
import numpy as np
np.random.seed(1235) # set both numpy and tf seeds, to make results reproducible. note this must be done for both.

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# set tf to print minimum

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
# allow tf to only print errors

import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow.keras import layers
tf.keras.backend.clear_session()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
# make tf use dynamic memory allocation on all gpus

tf.random.set_seed(1234)

import subprocess

import datetime

# import yaml

# import imageio
from PIL import Image

import glob

# from matplotlib import cm

import IPython.display

#from keras_tqdm import TQDMNotebookCallback # for jupyter notebooks

In [ ]:
import importlib
import sys
sys.path.append('/home/cstrauss2/Cancer16_LatentAttack/')
import Model
# contains my models for classifier, autoencoder...

In [ ]:
features = 36

In [ ]:
ae = Model.Autoencoder.get_model(features=features)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) # tf.keras.optimizers.SGD(learning_rate=0.001) #
ae.compile(
    optimizer=optimizer,
    loss='mse',
    metrics=['mse', 'binary_crossentropy']
)
ae_folder = '/home/cstrauss2/Cancer16_AE/models/11_12/08_00'
ae.load_weights(os.path.join(ae_folder, 'weights/weights'))

In [ ]:
classifier = Model.Classifier.get_model(features=features)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) # tf.keras.optimizers.SGD(learning_rate=0.001) #
classifier.compile(
    optimizer=optimizer,
    loss='mse',
    metrics=['mse', 'binary_crossentropy']
)
classifier_folder = '/home/cstrauss2/Cancer16_Classifier/models/01_21/11_15'
classifier.load_weights(os.path.join(classifier_folder, 'weights/weights'))

In [ ]:
def heatmap(tile):
    return classifier.predict(ae.encoder.predict(tile))

In [ ]:
tiles = np.sort(glob.glob('/home/cstrauss2/CycleGAN-Tensorflow-2/datasets/camelyon16normal2tumor/trainA/tumor_110*png'))
len(tiles)

In [ ]:
def get_img(fn):
    a = np.array(Image.open(fn))[:,:,:3].astype(np.float32)[np.newaxis]/255
#     print('Real Image', a.shape)
    return a * 2 - 1
def clip_img(in_arry):
    arry = np.copy(in_arry)
    return np.clip(arry*255, 0, 255).astype(np.uint8)

In [ ]:
import config

In [ ]:
config.args.dataset

In [ ]:
importlib.reload(config)

In [ ]:
config.args.dataset = 'camelyon16normal2tumor'
config.args.load_size = 512
config.args.crop_size = 512
config.args.identity_loss_weight = 5

In [ ]:
import train as cycleGAN

In [ ]:
test_img_A = get_img(tiles[1])
Image.fromarray(clip_img((test_img_A[0]+1)/2)).resize([256, 256])

In [ ]:
test_img_A.shape

In [ ]:
def recur(func, depth, args):
    if depth>0:
        return recur(func, depth-1, func(args))
    return args

In [ ]:
test_img_B = recur(cycleGAN.G_A2B, depth=2, args=(cycleGAN.G_A2B(test_img_A, training=False))).numpy()
test_img_B.shape

In [ ]:
np.min(test_img_A), np.max(test_img_A), np.min(test_img_B), np.max(test_img_B)

In [ ]:
delta = (test_img_A-test_img_B+1)/2
np.min(delta), np.max(delta), np.std(delta)

In [ ]:
delta_r = delta-np.min(delta)
np.min(delta_r), np.max(delta_r), np.std(delta_r)

In [ ]:
Image.fromarray(
    clip_img(
        np.concatenate(
            [(test_img_A+1)/2, (test_img_B+1)/2, (delta_r)],
            axis=2
        )[0]
    )
)

In [ ]:
Image.fromarray(
    clip_img(
        np.concatenate(
            [heatmap((test_img_A+1)/2), heatmap((test_img_B+1)/2), heatmap((delta_r)/np.max(delta_r))],
            axis=2
        )[0,:,:,0]
    )
)